In [3]:
! python -m pip install google-genai lxml numpy

   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/12.3 MB 9.3 MB/s eta 0:00:02
   ---------- ----------------------------- 3.1/12.3 MB 8.3 MB/s eta 0:00:02
   --------------- ------------------------ 4.7/12.3 MB 8.3 MB/s eta 0:00:01
   ---------------------- ----------------- 6.8/12.3 MB 8.8 MB/s eta 0:00:01
   ----------------------------- ---------- 9.2/12.3 MB 9.2 MB/s eta 0:00:01
   ------------------------------------- -- 11.5/12.3 MB 9.6 MB/s eta 0:00:01
   ---------------------------------------- 12.3/12.3 MB 9.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from lxml import etree
from google.genai import client, types
from google import genai
import numpy as np

api_key = ''
client = genai.Client(api_key=api_key)